In [1]:
import os
import sys
import subprocess
import datetime
import glob
import shutil

def main():
    # ========== 基本設定 ==========
    CONDA_PATH = r"C:\Users\100ca\anaconda3"
    ENV_NAME = "base"
    SCRIPTS_DIR = r"M:\ML\ChatGPT\gennote\batch\scripts"
    LOGS_DIR = r"M:\ML\ChatGPT\gennote\batch\batch_logs"
    PYTHON_EXE = os.path.join(CONDA_PATH, "python.exe")
    MAX_LOGS = 30

    # ========== 事前チェック ==========
    if not os.path.exists(CONDA_PATH):
        print(f"[ERROR] Conda path not found: {CONDA_PATH}")
        return 1

    if not os.path.exists(SCRIPTS_DIR):
        print(f"[ERROR] Scripts directory not found: {SCRIPTS_DIR}")
        return 1

    # ========== ログ管理 ==========
    if not os.path.exists(LOGS_DIR):
        try:
            os.makedirs(LOGS_DIR)
        except Exception as e:
            print(f"[ERROR] Failed to create logs directory: {e}")
            return 1

    # 古いログ削除（最新30件を保持）
    log_files = sorted(glob.glob(os.path.join(LOGS_DIR, "*.log")), key=os.path.getmtime, reverse=True)
    for old_log in log_files[MAX_LOGS:]:
        try:
            os.remove(old_log)
        except Exception as e:
            print(f"[WARNING] Failed to delete old log {old_log}: {e}")

    # ========== Conda環境設定 ==========
    # Windowsでの conda activate コマンドの実行方法
    if sys.platform == 'win32':
        activate_cmd = f'"{os.path.join(CONDA_PATH, "condabin", "conda.bat")}" activate {ENV_NAME}'
    else:
        activate_cmd = f'source "{os.path.join(CONDA_PATH, "bin", "activate")}" {ENV_NAME}'

    # ========== スクリプト実行 ==========
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_file = os.path.join(LOGS_DIR, f"{timestamp}.log")

    with open(log_file, 'w') as log:
        # search_*.py スクリプトを検索して実行
        search_scripts = glob.glob(os.path.join(SCRIPTS_DIR, "search_*.py"))
        
        for script in search_scripts:
            script_name = os.path.basename(script)
            current_time = datetime.datetime.now().strftime("%H:%M:%S")
            log.write(f"[{current_time}] START: {script_name}\n")
            log.flush()
            
            try:
                # Conda環境内でPythonスクリプトを実行
                cmd = f'{activate_cmd} && "{PYTHON_EXE}" "{script}"'
                process = subprocess.Popen(
                    cmd, 
                    shell=True, 
                    stdout=subprocess.PIPE, 
                    stderr=subprocess.STDOUT,
                    universal_newlines=True
                )
                
                # 出力をログファイルに書き込む
                for line in process.stdout:
                    log.write(line)
                    log.flush()
                
                process.wait()
                
                if process.returncode != 0:
                    log.write(f"[ERROR] FAILED: {script_name}\n")
                else:
                    log.write(f"[SUCCESS] COMPLETED: {script_name}\n")
                
            except Exception as e:
                log.write(f"[ERROR] Exception while running {script_name}: {str(e)}\n")
        
        # ========== 後処理 ==========
        if sys.platform == 'win32':
            deactivate_cmd = f'"{os.path.join(CONDA_PATH, "condabin", "conda.bat")}" deactivate'
        else:
            deactivate_cmd = f'source "{os.path.join(CONDA_PATH, "bin", "deactivate")}"'
        
        try:
            subprocess.run(deactivate_cmd, shell=True, check=True)
        except subprocess.CalledProcessError:
            log.write("[WARNING] Failed to deactivate conda environment\n")

    print(f"All processes completed. Log: {log_file}")
    return 0

if __name__ == "__main__":
    sys.exit(main())


All processes completed. Log: M:\ML\ChatGPT\gennote\batch\batch_logs\2025-03-24_20-47-06.log


SystemExit: 0

c:\Users\100ca\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
